In [11]:
import numpy as np
import cv2
import pandas as pd
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [31]:
#Read in file path for spreadsheet with lesion counts per body site for a patient
file_path = r'M:\\PALM\\REDCapReports\\BaselineLesionAssessment\\BaselineLesionAssessment_240329_SilaphetTyra.xlsx'
# Read in the spreadsheet with body site and lesion count data
data = pd.read_excel(file_path, sheet_name='BaselineLesionAssessment_240329')

# Select only the row with a certain ID
patient_id = "01-0004"  # Specify the patient ID you are interested in
single_patient_data = data[data['PID or Screening ID:'] == patient_id].head(1)

# Set the index name explicitly to match the patient ID
single_patient_data = single_patient_data.rename_axis(index=patient_id)
print(single_patient_data)

        PID or Screening ID:                       Event Name  \
01-0004                                                         
3                    01-0004  BASELINE (DAY 1) (Arm 1: Study)   

         Repeat Instrument  Repeat Instance  \
01-0004                                       
3                      NaN              NaN   

        1. Une évaluation des lésions de base a-t-elle été effectuée? Si oui, indiquez le nom-bre de lésions pour les parties du corps suivantes: Was a baseline lesion assessment completed? If yes, provide lesion counts for the following body sections:  \
01-0004                                                                                                                                                                                                                                                        
3                                       Oui (<i> Yes </I>)                                                                                            

In [32]:
#Combined R Axilla with R Anterior Arm, Combined L Axilla with L Anterior Arm
single_patient_data['CombinedR'] = single_patient_data['BLA1.1 Anterior arm (assessment region)'].astype(int) + single_patient_data['BLA1.3 Axilla']
single_patient_data['CombinedL'] = single_patient_data['BLA1.19 Anterior Arm (non-assessment region)'].astype(int) + single_patient_data['BLA1.21 Axilla (non-assessment region)']

#Dropped the unnecessary columns (dropped RAntArm, RAxilla, LAntArm, LAxilla)
lesion_data_cleancolumns = single_patient_data.drop(columns=['Event Name', 'Repeat Instrument', 'Repeat Instance', 
                                              '1. Une évaluation des lésions de base a-t-elle été effectuée? Si oui, indiquez le nom-bre de lésions pour les parties du corps suivantes: Was a baseline lesion assessment completed? If yes, provide lesion counts for the following body sections:', 
                                             'BLA1.1. Anterior arm (assessment region) ND', 'BLA1.2. posterior arm ND', 'BLA1.3. Axilla ND', 'BLA1.4. Palm ND', 'BLA1.5. Dorsal Hand ND', 'BLA1.6. Anterior Thigh ND', 'BLA1.7. Shin & Knee ND', 'BLA1.8. Posterior Thigh ND', 'BLA1.9. Calf & Back of knee ND',
                                             'BLA1.10. Dorsal Foot ND', 'BLA1.11. Sole ND', 'BLA1.12. Anterior Trunk ND', 'BLA1.13. Posterior Trunk ND', 'BLA1.14. Buttocks ND', 'BLA1.15 Face & Ears ND', 'BLA1.16. Anterior Neck ND', 'BLA1.17. Posterior Neck ND', 'BLA1.18. Scalp ND', 'BLA1.19. Anterior Arm (NAR) ND',
                                             'BLA1.20. Posterior Arm NAR ND', 'BLA1.21. Axilla NAR ND', 'BLA1.22. Palm NAR ND', 'BLA1.23. Dorsal Hand NAR ND', 'BLA1.24. Anterior Thigh NAR ND', 'BLA1.25. Skin & knee NAR ND', 'BLA1.26. Posterior Thigh NAR ND', 'BLA1.27. Calf & back of knee NAR ND', 'BLA1.28. Dorsal Foot NAR ND',
                                             'BLA1.29. Sole NAR ND', 'BLA1.30. Genitals ND', 'Complete?', 'BLA1.1 Anterior arm (assessment region)', 'BLA1.3 Axilla', 'BLA1.19 Anterior Arm (non-assessment region)', 'BLA1.21 Axilla (non-assessment region)'])
#print(lesion_data_cleancolumns.head())

#Renamed Columns to match M Drive 
lesion_data_cleancolumns.rename(columns = {'BLA1.2 Posterior Arm':'RightPosteriorArm', 
                                           'BLA1.4 palm':'RightPalm', 
                                           'BLA1.5. Dorsal Hand':'RightDorsalHand', 
                                           'BLA1.6 Anterior Thigh':'RightAnteriorThigh', 
                                           'BLA1.7 Shin & Knee':'RightShin&Knee', 
                                           'BLA1.8 Posterior Thigh':'RightPosteriorThigh',
                                           'BLA1.9 Calf & back of knee':'RightCalf&BackofKnee',
                                           'BLA1.10 Dorsal Foot':'RightDorsalFoot', 
                                           'BLA1.11 Sole':'RightSole', 
                                           'BLA1.12. Anterior Trunk':'AnteriorTrunk',
                                           'BLA1.13 Posterior Trunk':'PosteriorTrunk',
                                           'BLA1.14. Buttocks':'Buttocks',
                                           'BLA1.15 Face & Ears':'Face&Ears',
                                           'BLA1.16 Anterior Neck':'AnteriorNeck',
                                           'BLA1.17 Posterior Neck':'PosteriorNeck',
                                           'BLA1.18 Scalp':'Scalp',
                                           'BLA1.20 Posterior Arm (non-assessment region)':'LeftPosteriorArm',
                                           'CombinedR':'RightAnteriorArm',
                                           'CombinedL':'LeftAnteriorArm',
                                           'BLA1.22 Palm (non-assessment region)':'LeftPalm',
                                           'BLA1.23 Dorsal Hand (non-assessment region)':'LeftDorsalHand',
                                           'BLA1.24 Anterior thigh (non-assessment region)':'LeftAnteriorThigh',
                                           'BLA1.25 Skin & Knee (non-assessment region)':'LeftShin&Knee',
                                           'BLA1.26 Posterior Thigh (non-assessment region)':'LeftPosteriorThigh',
                                           'BLA1.27 Calf & back of knee (non-assessment region)':'LeftCalf&BackofKnee',
                                           'BLA1.28 Dorsal Foot (non-assessment region)':'LeftDorsalFoot',
                                           'BLA1.29 Sole (non-assessment region)':'LeftSole',
                                           'BLA1.30 Genitals':'Genitals'},
                              inplace = True)
#print(lesion_data_cleancolumns.head())

#Convert columns to rows
melted_df = lesion_data_cleancolumns.melt(id_vars=['PID or Screening ID:'], var_name='Body Site', value_name='Lesion Count')
print(melted_df)

   PID or Screening ID:             Body Site  Lesion Count
0               01-0004     RightPosteriorArm          38.0
1               01-0004             RightPalm           1.0
2               01-0004       RightDorsalHand          14.0
3               01-0004    RightAnteriorThigh          37.0
4               01-0004        RightShin&Knee          23.0
5               01-0004   RightPosteriorThigh          42.0
6               01-0004  RightCalf&BackofKnee          21.0
7               01-0004       RightDorsalFoot           3.0
8               01-0004             RightSole           0.0
9               01-0004         AnteriorTrunk          38.0
10              01-0004        PosteriorTrunk          72.0
11              01-0004              Buttocks          16.0
12              01-0004             Face&Ears          46.0
13              01-0004          AnteriorNeck           4.0
14              01-0004         PosteriorNeck           0.0
15              01-0004                 

In [33]:
from PIL import Image, ImageOps, ImageDraw
import pandas as pd
import os

data = melted_df

# Specify the folder paths for input and output images
input_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\FilledMasks\\AllSites\\'
output_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\'
overlay_folder = 'M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\'

# Color scale definition
# Have to make 0, 0.1 off white
color_scale = {
    (0, 0.1): "rgb(253, 253, 253)",
    (1, 50): "rgb(245, 171, 178)",
    (51, 100): "rgb(255, 2, 0)",
    (101, 150): "rgb(209, 1, 0)",
    (151, 200): "rgb(140, 0, 0)",
    (201, 250): "rgb(71, 0, 1)",
    (251, 500): "rgb(0, 0, 0)" 
}

# Function to map lesion count to color based on the color scale
def map_lesion_count_to_color(lesion_count):
    for range_, color in color_scale.items():
        if range_[0] <= lesion_count <= range_[1]:
            return color
    return "black"

# Iterate through each row in the spreadsheet
for index, row in data.iterrows():
    # Extract body site and lesion count
    body_site = row['Body Site']
    lesion_count = row['Lesion Count']

    # Construct the file path for the corresponding binary mask
    #HAS TO BE WHITE BG
    binary_mask_path = f'{input_folder}{body_site}_WhiteBG.png'
    
    # Open the corresponding binary mask
    binary_mask = Image.open(binary_mask_path).convert("L")
    
    # Map lesion count to color
    color = map_lesion_count_to_color(lesion_count)

    # Create a copy of the binary mask and fill in black pixels with the color
    colored_mask = ImageOps.colorize(binary_mask, black=color, white="white")
    
    # Ensure the colored mask has RGBA mode
    colored_mask = colored_mask.convert("RGBA")

    # Remove the background by setting alpha channel to 0 for white pixels
    data = colored_mask.getdata()
    new_data = [(r, g, b, 0) if r == 255 and g == 255 and b == 255 else (r, g, b, a) for r, g, b, a in data]
    colored_mask.putdata(new_data)

    # Save the colored mask without background
    output_path = f'{output_folder}{body_site}_ColoredOutput.png'
    colored_mask.save(output_path, 'PNG')
    print(f"Processed: {body_site}")

print("Done")

Processed: RightPosteriorArm
Processed: RightPalm
Processed: RightDorsalHand
Processed: RightAnteriorThigh
Processed: RightShin&Knee
Processed: RightPosteriorThigh
Processed: RightCalf&BackofKnee
Processed: RightDorsalFoot
Processed: RightSole
Processed: AnteriorTrunk
Processed: PosteriorTrunk
Processed: Buttocks
Processed: Face&Ears
Processed: AnteriorNeck
Processed: PosteriorNeck
Processed: Scalp
Processed: LeftPosteriorArm
Processed: LeftPalm
Processed: LeftDorsalHand
Processed: LeftAnteriorThigh
Processed: LeftShin&Knee
Processed: LeftPosteriorThigh
Processed: LeftCalf&BackofKnee
Processed: LeftDorsalFoot
Processed: LeftSole
Processed: Genitals
Processed: RightAnteriorArm
Processed: LeftAnteriorArm
Done


In [34]:
from PIL import Image

# List of image pairs: (first_image_path, second_image_path, output_path)
image_pairs = [
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\AnteriorTrunk_ColoredOutput.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPosteriorArm_ColoredOutput.png", 
     f'{overlay_folder}AnteriorTrunkPosteriorArm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\AnteriorTrunkPosteriorArm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPalm_ColoredOutput.png", 
     f'{overlay_folder}RightPalm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightPalm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightDorsalHand_ColoredOutput.png", 
     f'{overlay_folder}RightDorsalHand_Overlayed.png'), 
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightDorsalHand_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightAnteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}RightAnteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightAnteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightShin&Knee_ColoredOutput.png", 
     f'{overlay_folder}RightShin&Knee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightShin&Knee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightPosteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}RightPosteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightPosteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightCalf&BackofKnee_ColoredOutput.png", 
     f'{overlay_folder}RightCalf&BackofKnee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightCalf&BackofKnee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightDorsalFoot_ColoredOutput.png", 
     f'{overlay_folder}RightDorsalFoot_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightDorsalFoot_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightSole_ColoredOutput.png", 
     f'{overlay_folder}RightSole_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightSole_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\RightAnteriorArm_ColoredOutput.png", 
     f'{overlay_folder}RightAnteriorArm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\RightAnteriorArm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\PosteriorTrunk_ColoredOutput.png", 
     f'{overlay_folder}PosteriorTrunk_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\PosteriorTrunk_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\Buttocks_ColoredOutput.png", 
     f'{overlay_folder}Buttocks_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\Buttocks_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\Face&Ears_ColoredOutput.png", 
     f'{overlay_folder}Face&Ears_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\Face&Ears_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\AnteriorNeck_ColoredOutput.png", 
     f'{overlay_folder}AnteriorNeck_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\AnteriorNeck_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\PosteriorNeck_ColoredOutput.png", 
     f'{overlay_folder}PosteriorNeck_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\PosteriorNeck_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\Scalp_ColoredOutput.png", 
     f'{overlay_folder}Scalp_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\Scalp_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftAnteriorArm_ColoredOutput.png", 
     f'{overlay_folder}LeftAnteriorArm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftAnteriorArm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftPosteriorArm_ColoredOutput.png", 
     f'{overlay_folder}LeftPosteriorArm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftPosteriorArm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftPalm_ColoredOutput.png", 
     f'{overlay_folder}LeftPalm_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftPalm_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftDorsalHand_ColoredOutput.png", 
     f'{overlay_folder}LeftDorsalHand_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftDorsalHand_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftAnteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}LeftAnteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftAnteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftShin&Knee_ColoredOutput.png", 
     f'{overlay_folder}LeftShin&Knee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftShin&Knee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftPosteriorThigh_ColoredOutput.png", 
     f'{overlay_folder}LeftPosteriorThigh_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftPosteriorThigh_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftCalf&BackofKnee_ColoredOutput.png", 
     f'{overlay_folder}LeftCalf&BackofKnee_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftCalf&BackofKnee_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftDorsalFoot_ColoredOutput.png", 
     f'{overlay_folder}LeftDorsalFoot_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftDorsalFoot_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\LeftSole_ColoredOutput.png", 
     f'{overlay_folder}LeftSole_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\LeftSole_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\ColoredMaskOutput\\Genitals_ColoredOutput.png", 
     f'{overlay_folder}Genitals_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\UnfilledMasks\\Blank\\AllSites_AllWhite.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\Genitals_Overlayed.png", 
     f'{overlay_folder}WhiteBG_Overlayed.png'),
    
    (r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\OverlayOutput\\WhiteBG_Overlayed.png", 
     r"M:\\Skin\\Data\\Monkeypox\\GuidelineDevelopment\\HeatMaps\\UnfilledMasks\\Blank\\AllSites_NoBG.png", 
     f'{overlay_folder}{patient_id}_Baseline.png'), 
    
]

# Iterate through each image pair
for first_image_path, second_image_path, output_path in image_pairs:
    # Loading the first image (with transparent BG)
    img1 = Image.open(first_image_path).convert("RGBA")

    # Loading the second image (with transparent BG)
    img2 = Image.open(second_image_path).convert("RGBA")
    img2 = img2.resize(img1.size)

    # Using alpha_composite to overlay the images onto each other
    img3 = Image.alpha_composite(img1, img2)
    img3.save(output_path, 'PNG')
    
img3.show()

print("Done")


Done
